In [ ]:
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/patches/1/cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
# !dpkg -i cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
# !apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
# !apt-get update
# !apt-get install cuda=9.2.88-1
# !dpkg -i cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
# !apt-get install nvidia-cuda-toolkit

In [ ]:
# !nvcc --version

In [ ]:
# !pip install torch==1.4.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install torch-scatter==latest+cu92 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
# !pip install torch-sparse==latest+cu92 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
# !pip install torch-cluster==latest+cu92 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
# !pip install torch-spline-conv==latest+cu92 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
# !pip install torch-geometric

## Install suitable versions of Cuda and Torch:

In [ ]:
!pip install torch==1.5.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


## Import libraries for Geometric Deep Learning (Pytorch Geometric):

In [ ]:
import numpy as np
import os.path as osp
import torch
import torch.nn.functional as F
import pandas as pd
import torch_geometric.nn as pyt_geom
from torch_geometric.nn import SplineConv
from torch_geometric.data import Data
from random import shuffle, randint
import networkx as nx
import matplotlib.pyplot as plt
import random 

import scipy.sparse as scipy
from torch_sparse import coalesce
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from torch_geometric.data import Batch
from networkx import parse_edgelist 

# **1) Load biomedical datasets for 5 tissues:**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

meth_files = ['Hom_Normal_Liver_gene_meth_profiles', 'Hom_Healthy_Lung_gene_meth_profiles', 'Hom_Normal_Pancreas_34yrs_gene_meth_profiles', 'Hom_Normal_AdrenalGland_34yrs_gene_meth_profiles', 'Hom_Healthy_Spleen_3yrs_gene_meth_profiles']
RNA_files = ['Liver', 'Lung', 'Pancreas', 'Adrenal_Gland', 'Spleen']
GGI_files = ['liver.dat', 'lung.dat', 'pancreas.dat', 'adrenal_gland.dat', 'spleen.dat']

methylation_list = []
RNA_list = []
GGI_list = []

for meth_name, RNA_name, GGI_name in zip(meth_files, RNA_files, GGI_files):
  meth_df = pd.read_csv("/gdrive/My Drive/All_data/methylation/"+file_name, delimiter="\t")
  RNA_df = pd.read_csv("/gdrive/My Drive/All_data/RNA-seq/"+RNA_name+".v8.normalized_expression.bed",  delimiter="\t") 
  GGI_df = pd.read_csv("/gdrive/My Drive/All_data/GGI_networks/"+GGI_name,  delimiter="\t") 
  
  methylation_list.append(df)
  RNA_list.append(RNA_df)
  GGI_list.append(GGI_df)

# Gene mapping between Ensembl ID and Entrez ID
gene_mapping = pd.read_csv("/gdrive/My Drive/R Gene-ID mapping/Gtex_mapping.csv")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# GGI_list[0].head()

In [ ]:
# RNA_list[4].head()

In [ ]:
# methylation_list[0].head()

# **2) Data Pre-processing:**

## Gene ID mapping:

In [ ]:
def create_gene_id_map(gene_mapping):

    gene_id_map = {} # Ensembl_gene_ID --> Entrez_ID

    for ensembl_id, entrez_id in zip(gene_mapping['Ensembl_gene_ID'], gene_mapping['Entrez_ID']):
        if(not np.isnan(entrez_id)):
            gene_id_map[ensembl_id] = int(entrez_id)
    
    return gene_id_map

gene_id_map = create_gene_id_map(gene_mapping)
list(gene_id_map.items())[:5]

[('ENSG00000000003', 7105),
 ('ENSG00000000005', 64102),
 ('ENSG00000000419', 8813),
 ('ENSG00000000457', 57147),
 ('ENSG00000000460', 55732)]

## DNA Methylation data processing:

In [ ]:
# Dictionary to map each (gene_id, tissue_id) to its methylation data
def get_methylation_dic(methylation_list):

    methylation_dic = {}

    # Iterate over methylation data for each tissue in the list
    for tissue_index, meth_df in enumerate(methylation_list): 
      
        # Renaming mehtylation data and selecting columns of interest
        meth_data = meth_df.copy()
        meth_data = meth_data.iloc[: , [1, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
        meth_data.columns = ["Gene", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9"]  

        new_gene_ids = [gene_id_map.get(meth_data['Gene'][i].strip(), None) for i in range(len(meth_data))]
        
        # The genes are now encoded with the entrez_id
        meth_data["Gene"] = new_gene_ids

        meth_data = meth_data.dropna()

        # if multiple "Ensembl ID" genes were encoded to the same "Entrez_ID" gene, keep the first entry
        meth_data.drop_duplicates(subset ="Gene", keep = 'first', inplace = True)
        
        for idx, row in meth_data.iterrows():
            gene = int(row.values[0])
            vals = row.values[1:]
            new_vals = [0 if v==' na ' else round(float(v), 7) for v in vals]
            
            # map each (gene_id, tissue_id) to its 9D methylation vector
            methylation_dic[(gene, tissue_index)] = new_vals
            

    return methylation_dic
      

methylation_dic = get_methylation_dic(methylation_list)

## RNA-seq data processing:

In [ ]:
# Dictionary to map each (gene_id, tissue_id) to its RNA data
RNA_dic = {}

def get_RNA_dic(RNA_list):

    new_RNA_list = []

    for tissue_index, tiss_df in enumerate(RNA_list):
        tiss_data = tiss_df.copy()
        tiss_data.drop(['#chr', 'start', 'end'], inplace=True, axis=1)
        gene_list = [v.split('.', 1)[0] for v in tiss_data.gene_id]
        tiss_data.gene_id = [gene_id_map.get(ensembl_gene, None) for ensembl_gene in gene_list] # genes are now encoded with the entrez_id
        tiss_data = tiss_data.dropna() # drop rows where genes can't be mapped 
        tiss_data.drop_duplicates(subset ="gene_id", keep = 'first', inplace = True)
        new_RNA_list.append(tiss_data)
        
        for idx, row in tiss_data.iterrows():
            gene = int(row.values[0])
            vals = row.values[1:]
            
            # map each (gene_id, tissue_id) to its 208D RNA vector
            RNA_dic[(gene, tissue_index)] = vals

    return RNA_dic, new_RNA_list

RNA_dic, new_RNA_list = get_RNA_dic(RNA_list)

### Rename columns for Gene-Gene-Interaction(GGI) data:

In [ ]:
def rename_GGI_columns():
    for index, GGI in enumerate(GGI_list):
        GGI['y'] = index
        GGI.columns = ['gene1', 'gene2', 'edge_type', 'y'] 

rename_GGI_columns()

In [ ]:
print(len(list(RNA_dic.keys())))
print(len(list(methylation_dic.keys())))

89794
114435


## Functions to create Pytorch Geometric Data Object

In [ ]:
def get_features(unique_gene_ids, tissue_label, feat_dict):
    num_features = 0
    for key in feat_dict.keys():
        if key[1]==tissue_label:
            num_features = len(feat_dict[key])
            break

    features = []
    for gene in list(unique_gene_ids): 
        if ((gene, tissue_label) in feat_dict.keys()):
            features.append(np.array(feat_dict[(gene,tissue_label)]).reshape(-1,1))
        else:
            features.append(np.zeros(shape=(num_features,1)))

    features = np.array(features)
    features = features.reshape((features.shape[0], features.shape[1]))

    return torch.as_tensor(features, dtype=torch.float)

In [ ]:
# Assign new ID to gene nodes, from 0 to n
def update_GGI_data(GGI_df, sorted_uniq_gene_ids):
    new_GGI_df = GGI_df.copy()
    idx_idx_dic = {}
    for index, original_index in enumerate(sorted_uniq_gene_ids):
        idx_idx_dic[original_index] = index
    
    new_GGI_df['gene1'] = new_GGI_df.gene1.map(idx_idx_dic)
    new_GGI_df['gene2'] = new_GGI_df.gene2.map(idx_idx_dic)
    
    return new_GGI_df

In [ ]:
GGI_data_list = []

# For each Gene-Gene-Interaction(GGI) network
for ggi_df in GGI_list:
    ggi_df = ggi_df[ggi_df['edge_type']== 1]
    uniq_gene_ids = list(set(list(ggi_df.gene1) + list(ggi_df.gene2)))
    tissue_label = ggi_df.y.unique()[0] 

    # Get methylation node features
    meth_feat = get_features(uniq_gene_ids, tissue_label, methylation_dic)
    
    # Get RNA node features
    RNA_feat = get_features(uniq_gene_ids, tissue_label, RNA_dic)

    # Get new GGI dataframe
    new_ggi_df = update_GGI_data(ggi_df, uniq_gene_ids)

    # Get edges from GGI dataframe
    edge_index = torch.as_tensor(new_ggi_df[['gene1', 'gene2']].values.T, dtype=torch.long)
    
    # Get labels
    y = new_ggi_df.y.unique()
    y = np.ones(shape=(len(uniq_gene_ids), 1))*y
    y = torch.as_tensor(y, dtype=torch.long).squeeze()

    # creating features filled with ones - equivalent to no features
    no_feat = torch.as_tensor(np.ones(shape=(len(uniq_gene_ids), 1)), dtype=torch.float)

    # Final Data object
    ggi_data = Data(no_feat=no_feat, RNA_feat=RNA_feat, meth_feat=meth_feat, y=y, edge_index=edge_index)
    GGI_data_list.append(ggi_data)

### **Final Data Objects:** 5 tissue-specific networks

In [ ]:
min_RNA_feat = np.min([GGI_data_list[0].RNA_feat.shape[1], GGI_data_list[1].RNA_feat.shape[1], GGI_data_list[2].RNA_feat.shape[1], GGI_data_list[3].RNA_feat.shape[1], GGI_data_list[4].RNA_feat.shape[1]])

for data_obj in GGI_data_list:
    data_obj.RNA_feat = data_obj.RNA_feat[:, :min_RNA_feat]

# Final list of Gene-Gene-Interaction(GGI) networks with RNA and methylation node features.
GGI_data_list

[Data(RNA_feat=[3315, 208], edge_index=[2, 81684], meth_feat=[3315, 9], no_feat=[3315, 1], y=[3315]),
 Data(RNA_feat=[3339, 208], edge_index=[2, 95722], meth_feat=[3339, 9], no_feat=[3339, 1], y=[3339]),
 Data(RNA_feat=[3169, 208], edge_index=[2, 70364], meth_feat=[3169, 9], no_feat=[3169, 1], y=[3169]),
 Data(RNA_feat=[2366, 208], edge_index=[2, 16741], meth_feat=[2366, 9], no_feat=[2366, 1], y=[2366]),
 Data(RNA_feat=[3058, 208], edge_index=[2, 65110], meth_feat=[3058, 9], no_feat=[3058, 1], y=[3058])]

The nodes indices were redefined in the range [0, num_nodes]. This enables each index in edge_index to access the corresponding feature vector in RNA_feat and meth_feat.

This is a test to make sure the mapping from old indices to new indices is correct. In this example 285590 is the old index and 2990 is the new index. 

In [ ]:
print(GGI_data_list[0].meth_feat[2990] == torch.as_tensor(methylation_dic[(285590,0)]))

tensor([True, True, True, True, True, True, True, True, True])


In [ ]:
def get_feature_coverage(GGI_data_list, tissue_index):
    data = GGI_data_list[tissue_index]
    n = len(data.y)
    RNA_counter = 0
    meth_counter = 0

    for i in range(n):
        meth_feat = data.meth_feat[i]
        RNA_feat = data.RNA_feat[i]

        if RNA_feat.float().sum() == 0:
            RNA_counter += 1

        if meth_feat.float().sum() == 0:
            meth_counter += 1

    RNA_node_coverage = np.round((1 - RNA_counter/n), 4)
    meth_node_coverage = np.round((1 - meth_counter/n), 4)
    
    return RNA_node_coverage, meth_node_coverage


for tissue_index in range(len(GGI_data_list)):
    RNA_node_coverage, meth_node_coverage = get_feature_coverage(GGI_data_list, tissue_index)
    print("TISSUE NETWORK ", tissue_index, ":")
    print("RNA data covers ", RNA_node_coverage*100, "% of the gene nodes")
    print("Methylation data covers ", meth_node_coverage*100, "% of the gene nodes")
    print("\n")

TISSUE NETWORK  0 :
RNA data covers  94.81 % of the gene nodes
Methylation data covers  99.58 % of the gene nodes


TISSUE NETWORK  1 :
RNA data covers  98.56 % of the gene nodes
Methylation data covers  99.58 % of the gene nodes


TISSUE NETWORK  2 :
RNA data covers  97.7 % of the gene nodes
Methylation data covers  99.56 % of the gene nodes


TISSUE NETWORK  3 :
RNA data covers  97.89 % of the gene nodes
Methylation data covers  99.53999999999999 % of the gene nodes


TISSUE NETWORK  4 :
RNA data covers  98.3 % of the gene nodes
Methylation data covers  99.57000000000001 % of the gene nodes




# **3) Link prediction with a Variational Graph Auto-Encoder (VGAE):**

In this section, we train the VGAE to achieve link prediction after combining biological features (RNA+Methylation) in the latent space.

1.   We train two separate GCN encoders (Graph Convolutional Neural Network). One is trained on a GGI network enriched with methylation features and the other is trained on a GGI network enriched with RNA features.
2.   The two encoded representations are then concatenated in the latent space and fed to the decoder of the VGAE which performs link prediction (i.e graph reconstruction).



In [ ]:
import networkx as nx
import torch_geometric
from scipy.sparse import coo_matrix, hstack, vstack

# choosing the Gene-Gene-Interaction network
data = GGI_data_list[1]

graph = nx.Graph()
edgeinfo = data.edge_index
src = edgeinfo[0].cpu().numpy()
dst = edgeinfo[1].cpu().numpy()
edgelist = zip(src,dst)
for i,j in edgelist:
  graph.add_edge(i,j) 

A = nx.adjacency_matrix(graph) # ADJACENCY MATRIX USED IN TRAINING

non_edges = list(nx.non_edges(graph))
e0 = [tup[0] for tup in non_edges]
e1 = [tup[1] for tup in non_edges]
neg_edge_list = np.vstack((e0, e1))
neg_edge_index = torch.as_tensor(neg_edge_list, dtype=torch.long)

w_factor = (neg_edge_index.shape[1]/edgeinfo.shape[1])

## Build the Encoder, Decoder and VGAE class:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch_geometric.nn as pyg_nn


class GraphConvolution(nn.Module):
    """Encoder which maps the network into a low-dimensional space."""

    def __init__(self, in_features, out_features, dropout=0., act=F.relu):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        input = F.dropout(input, self.dropout, self.training)
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        output = self.act(output)
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


class InnerProductDecoder(nn.Module):
    """Decoder which uses inner product for link prediction."""

    def __init__(self, dropout, act=torch.sigmoid):
        super(InnerProductDecoder, self).__init__()
        self.dropout = dropout
        self.act = act

    def forward(self, z):
        z = F.dropout(z, self.dropout, training=self.training)
        adj = self.act(torch.mm(z, z.t()))
        return adj




class GCNModelVAE(nn.Module): 
    """VGAE model which includes an encoder and a decoder."""

    def __init__(self, RNA_feat_dim, methy_feat_dim, hidden_dim1, hidden_dim2, dropout):
        super(GCNModelVAE, self).__init__()
        self.gcR = GraphConvolution(RNA_feat_dim, hidden_dim1, dropout, act=F.relu)
        self.gcM = GraphConvolution(methy_feat_dim, hidden_dim1, dropout, act=F.relu)
        self.gc2 = GraphConvolution(hidden_dim1*2, hidden_dim2, dropout, act=lambda x: x)
        self.gc3 = GraphConvolution(hidden_dim1*2, hidden_dim2, dropout, act=lambda x: x)
        self.dc = InnerProductDecoder(dropout, act=lambda x: x)

    def encode(self, x_RNA, x_methy, adj):
        hidden1_RNA = self.gcR(x_RNA, adj)
        hidden1_methy = self.gcM(x_methy, adj)
        hidden1 = torch.cat((hidden1_RNA, hidden1_methy), 1)
        return self.gc2(hidden1, adj), self.gc3(hidden1, adj)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x_RNA, x_methy, adj):
        mu, logvar = self.encode(x_RNA, x_methy, adj)
        z = self.reparameterize(mu, logvar)
        return self.dc(z), mu, logvar



def loss_function(preds, labels, mu, logvar, n_nodes, norm, pos_weight):
    cost = norm * F.binary_cross_entropy_with_logits(preds, labels, pos_weight=pos_weight)
    KLD = -0.5 / n_nodes * torch.mean(torch.sum(1 + 2 * logvar - mu.pow(2) - logvar.exp().pow(2), 1))
    return cost + KLD

## Define pre-processing functions:

In [ ]:
import pickle as pkl

import networkx as nx
import numpy as np
import scipy.sparse as sp
import torch
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape


def mask_test_edges(adj):
    # Function to build a test set and validation set, each with 10% of the positive links.
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.))
    num_val = int(np.floor(edges.shape[0] / 10.))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)


    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])


    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false


def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return sparse_mx_to_torch_sparse_tensor(adj_normalized)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


## Get performance scores:

In [ ]:
def get_scores(emb, adj_orig, edges_pos, edges_neg):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])

    p = np.array([1 if  p>0.5 else 0 for p in preds_all]).reshape((-1,1))
    y = labels_all.reshape((-1,1))
    c = confusion_matrix(y, p, normalize='true')
    tn, fp, fn, tp = c.ravel()
    balanced_acc = (tp+tn)/2
    f1_sco = f1_score(y, p, average='weighted')

    return balanced_acc, f1_sco

In [ ]:
# ADD confusion matrices et precision-recall curves for evaluation
# Enable GPU

## Train the VGAE model:

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import scipy.sparse as sp
import torch
from torch import optim

seed = int(42)
hidden1 = int(32)
hidden2 = int(16)
lr = 0.01
dropout = np.float(0.2)
epochs = 1500


def gae_for():
    adj = A
    RNA_features = torch.as_tensor(data.RNA_feat, dtype=torch.float)
    methy_features = torch.as_tensor(data.meth_feat, dtype=torch.float)

    n_nodes = RNA_features.shape[0]
    RNA_feat_dim = RNA_features.shape[1]
    methy_feat_dim = methy_features.shape[1]

    # Store original adjacency matrix (without diagonal entries) for later
    adj_orig = adj
    adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
    adj_orig.eliminate_zeros()

    adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
    adj = adj_train

    # Some preprocessing
    adj_norm = preprocess_graph(adj)
    adj_label = adj_train + sp.eye(adj_train.shape[0])
    adj_label = torch.FloatTensor(adj_label.toarray())

    w = np.ones(adj_label.shape)
    w = w*w_factor
    weight = torch.as_tensor(w, dtype=torch.float)

    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

    model = GCNModelVAE(RNA_feat_dim, methy_feat_dim, hidden1, hidden2, dropout)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    hidden_emb = None

    for epoch in range(epochs):
        t = time.time()
        model.train()
        optimizer.zero_grad()
        recovered, mu, logvar = model(RNA_features, methy_features, adj_norm)
        loss = loss_function(preds=recovered, labels=adj_label,
                             mu=mu, logvar=logvar, n_nodes=n_nodes,
                             norm=norm, pos_weight=weight)
        loss.backward()
        cur_loss = loss.item()
        optimizer.step()

        hidden_emb = mu.data.numpy()

        val_bal_acc, val_f1_score = get_scores(hidden_emb, adj_orig, val_edges, val_edges_false)

        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss),
              "val_bal_acc=", "{:.5f}".format(val_bal_acc),
              "val_f1_score=", "{:.5f}".format(val_f1_score),
              "time=", "{:.5f}".format(time.time() - t)
              )

    print("Optimization Finished!")

    test_bal_acc, test_f1_score = get_scores(hidden_emb, adj_orig, test_edges, test_edges_false)
    print('Test Bal_acc: ' + str(test_bal_acc))
    print('Test F1_score: ' + str(test_f1_score))


if __name__ == '__main__':
    gae_for()